In [5]:
import os
import numpy as np
import pandas as pd
import pickle
from sksurv.svm import FastKernelSurvivalSVM
from sklearn.metrics.pairwise import pairwise_kernels


In [7]:
alpha=0.05
gamma=0.5
degree=3
beta=0.8
random_state=0

def gramMatrix(X1, X2, K_function):
    gram_matrix = np.zeros((X1.shape[0], X2.shape[0]))
    for i, x1 in enumerate(X1):
        for j, x2 in enumerate(X2):
            gram_matrix[i, j] = K_function(x1, x2)
    return gram_matrix

kernels = {
    'power': lambda x: gramMatrix(x, x, lambda x1, x2: np.clip(-np.linalg.norm(x1 - x2)**beta, 2e-100, 2e100)),
    'log': lambda x:  gramMatrix(x, x, lambda x1, x2: np.clip(-np.log(1 + np.linalg.norm(x1 - x2, axis=0)**beta), 2e-100, 2e100)),
    'mixture': lambda x: np.clip(alpha * pairwise_kernels(x, metric='rbf', gamma=gamma) + (1-alpha)*pairwise_kernels(x, metric='poly', degree=degree), 2e-100, 2e100),
}

In [2]:
MODELS_DIR='models'
# load model
model_file = open(os.path.join(MODELS_DIR, 'dataset_reduced.csv_SIFT_.npy_log_26-01-2023 06-55-40_model.sav'), 'rb')
model = pickle.load(model_file)
print(model.score)



0.5


In [3]:
print(model.cv_results)

{'fit_time': array([0.14061165, 0.09375715, 0.07812738, 0.07813907]), 'score_time': array([0.        , 0.01561785, 0.        , 0.01562166]), 'test_score': array([0.5, 0.5, 0.5, 0.5]), 'train_score': array([0.5, 0.5, 0.5, 0.5])}


In [4]:
dataset = np.load(os.path.join('features', 'dataset_reduced.csv_SIFT_.npy'), allow_pickle=True)
len(dataset)

1091

In [8]:
x = np.delete(dataset, [-3, -2, -1], axis=1).astype(np.float16)

In [11]:
model.coef_ == None

True

In [9]:
y = model.predict(x)
y

AttributeError: 'FastKernelSurvivalSVM' object has no attribute 'fit_X_'